<a href="https://colab.research.google.com/github/bryandaetz1/CA_brewery_map/blob/main/brewery_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install geopandas
!pip install geopy

     |████████████████████████████████| 972kB 6.5MB/s 
     |████████████████████████████████| 6.5MB 19.6MB/s 
     |████████████████████████████████| 14.8MB 299kB/s 


In [60]:
import pandas as pd
import geopandas as gpd
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

In [61]:
brew_df = pd.read_csv('https://raw.githubusercontent.com/bryandaetz1/CA_brewery_map/main/breweries_raw.csv')
beer_df = pd.read_csv('https://raw.githubusercontent.com/bryandaetz1/CA_brewery_map/main/beers_raw.csv')

In [62]:
#filling in missing or incorrect values based on actual values from website
brew_df.iloc[3] = ['Ohana Brewing Company',
                   '7 S 1st Street\nAlhambra, California, United States',
                   'http://ohanabrew.com']

brew_df.iloc[279] = ["The Dudes' Brewing Company",
                     '304\n395 Santa Monica Place\nSanta Monica, California, United States',
                     'http://www.thedudesbrew.com']

brew_df.iloc[280] = ['Cooperage Brewing Company',
                     'G\n981 Airway Court\nSanta Rosa, California, United States',
                     'https://cooperagebrewing.com']

brew_df.iloc[288] = ['Culture Brewing Co.',
                     'Ste 200\n111 S Cedros Ave\nSolana Beach, California, United States',
                     'https://culturebrewingco.com']

brew_df.iloc[305] = ['Back Street Brewery',
                     '#B100\n15 Main St.\nVista, California, United States',
                     'http://www.backstreetbrew.com']

brew_df.iloc[317] = ['MadeWest Brewing Company',
                     '1744 Donlon Street\nWestlake Village, California, United States',
                     'https://madewest.com']

In [63]:
#removing Address from the end of each brewery name in brew_df
brew_df['Brewery'] = brew_df['Brewery'].str.replace('Address','').str.strip()

#removing Address from the end of each brewery name in beer_df
beer_df['Brewery'] = beer_df['Brewery'].str.replace('Address','').str.strip()

In [64]:
#appending missing beers to beers_df dataframe
beers_append = pd.DataFrame(data = [['Ohana Brewing Company','Spa Water Saison','Saison / Farmhouse Ale'],
                                     ["The Dudes' Brewing Company",'Blood Orange','American Amber / Red Ale'],
                                     ["The Dudes' Brewing Company",'Boysenberry','American Pale Wheat Ale'],
                                     ["The Dudes' Brewing Company",'CalifornIPA','American IPA'],
                                     ["The Dudes' Brewing Company",'Coconut','American Porter'],
                                     ["The Dudes' Brewing Company",'Double Trunk','American IPA'],
                                     ["The Dudes' Brewing Company","Grandma's Pecan",'English Brown Ale'],
                                     ["The Dudes' Brewing Company","Los Dude's Cerveza",'Light Lager'],
                                     ["The Dudes' Brewing Company",'Most Excellent IPA','American IPA'],
                                     ["The Dudes' Brewing Company",'Peach','Berliner Weissbier'],
                                     ["The Dudes' Brewing Company",'Schitzengiggle','Märzen / Oktoberfest'],
                                     ["The Dudes' Brewing Company",'Wiser Dude','American Double / Imperial Pilsner'],
                                     ['Cooperage Brewing Company','Cultivating Mass','American Double / Imperial Stout'],
                                     ['Culture Brewing Co.','2x Black IPA','American Double / Imperial IPA'],
                                     ['Culture Brewing Co.','2x IPA','American Double / Imperial IPA'],
                                     ['Culture Brewing Co.', '3x IPA','American Double / Imperial IPA'],
                                     ['Culture Brewing Co.', 'Blond Ale','American Blonde Ale'],
                                     ['Back Street Brewery','Tomahawk Double IPA','American Double / Imperial IPA'],
                                     ['Back Street Brewery','Set Sail IPA','American IPA'],
                                     ['Back Street Brewery','Rita Red Ale','American Amber / Red Ale'],
                                     ['Back Street Brewery','Rydin’ Dirty Rye IPA','American IPA'],
                                     ['MadeWest Brewing Company','Donlon IIPA','American Double / Imperial IPA']
                                     ],
                            columns = ['Brewery','Beer','Style'])

beer_df = pd.concat([beer_df,beers_append], axis = 0, sort = False).reset_index(drop = True)

In [16]:
#geocoding brew_df to get longitude and latitude

locator = Nominatim(user_agent='bdaetz19@hotmail.com')

#function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

#create location column
brew_df['location'] = brew_df['Address'].apply(geocode)

#create longitude,latitude and altitude from location column (returns tuple)
brew_df['point'] = brew_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

#split point column into latitude, longitude and altitude columns
brew_df[['latitude','longitude','altitude']] = pd.DataFrame(brew_df['point'].tolist(), index = brew_df.index)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('6421 Central Avenue\nLittle River, California, United States',), **{}).
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/geopy/geocoders/base.py", line 344, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/usr/lib/python3.6/urllib/request.py", line 526, in open
    response = self._open(req, data)
  File "/usr/lib/python3.6/urllib/request.py", line 544, in _open
    '_open', req)
  File "/usr/lib/python3.6/urllib/request.py", line 504, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.6/urllib/request.py", line 1368, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/usr/lib/python3.6/urllib/request.py", line 1328, in do_open
    r = h.getresponse()
  File "/usr/lib/python3.6/http/client.py", line 1373, in getresponse
    response.begin()
  File "/usr/lib/python3.6/http/client.py", line 311, in begin
    ve

In [17]:
brew_df.head()

,Brewery,Address,Contact_Info,location,point,latitude,longitude,altitude
0,Alameda Island Brewing Company,"1716 Park Street\nAlameda, California, United ...",http://alamedaislandbrewingcompany.com/,"(1716, Park Street, Alameda, Alameda County, C...","(37.768997366838015, -122.23864316651895, 0.0)",37.768997,-122.238643,0.0
1,Faction Brewing,"2501 Monarch Street\nAlameda, California, Unit...",http://factionbrewing.com/,"(Faction Brewing, 2501, Monarch Street, Alamed...","(37.78651979999999, -122.30934879928142, 0.0)",37.786520,-122.309349,0.0
2,Ocean View Brew Works,"627 San Pablo Avenue\nAlbany, California, Unit...",https://www.oceanviewbrews.com/,"(Ocean View Brew Works, 627, San Pablo Avenue,...","(37.8941775, -122.2999572, 0.0)",37.894177,-122.299957,0.0
3,Ohana Brewing Company,"7 S 1st Street\nAlhambra, California, United S...",http://ohanabrew.com,"(7, 1st Street, Alhambra, Los Angeles County, ...","(34.09450469387755, -118.12790740816325, 0.0)",34.094505,-118.127907,0.0
4,Alpine Beer Company,"2363 Alpine Blvd\nAlpine, California, United S...",http://alpinebeerco.com/home/,"(2363, Alpine Boulevard, San Diego County, Cal...","(32.835121, -116.7626188, 0.0)",32.835121,-116.762619,0.0


In [19]:
brew_df.isna().sum()

Brewery           0
Address           0
Contact_Info      0
location        102
point           102
latitude        102
longitude       102
altitude        102
dtype: int64

In [26]:
#saving dataframe with the addresses that were successfully geocoded
brew_geocoded = brew_df[~brew_df['latitude'].isna()][['Brewery','Address','Contact_Info','latitude','longitude']].reset_index(drop = True)

In [27]:
brew_geocoded.to_csv('brew_geocoded_1.csv',index = False)

In [56]:
#saving dataframe with the addresses that were not successfully geocoded
brew_2 = brew_df[brew_df['latitude'].isna()][['Brewery','Address','Contact_Info']].reset_index(drop=True)

In [58]:
brew_2.to_csv('brew_uncoded.csv',index = False)

In [66]:
#creating csv file from the cleaned beers_df dataframe
beer_df.to_csv('beers.csv',index = False)